Give the start and end point in UNIX

In [1]:
import requests
import json
import time
import sys
import pandas as pd
import calendar
from datetime import datetime, timedelta

In [2]:
# date-to-UNIX function
def get_unix_ms_from_date(date):
    return int(calendar.timegm(date.timetuple()) * 1000 + date.microsecond/1000)


def get_ohlc_data(interval, symbol, start, end):
    r = requests.get(f'https://api-pub.bitfinex.com/v2/candles/trade:{interval}:t{symbol}/hist?limit=10000&start={start}&end={end}&sort=1')
    return r.json()

def fetch_data_from_bitfinex(interval, symbol, start, end):
    df = pd.DataFrame()
    
    while start < end:
        periodic_end = start + 601620000
        data = get_ohlc_data(interval, symbol, start, periodic_end)
        start = data[-1][0]
        trade_df = pd.DataFrame(data, columns = ['timestamp', 'Open', 'Close', 'High', 'Low', 'Volume'])
        trade_df = trade_df.set_index('timestamp')
        df = pd.concat([df, trade_df])
        df = df[~df.index.duplicated()]
        b = f"Status Start: {start}" 
        print(b, end="\r")
        time.sleep(0.1)   
    return df


def update_daily(df, symbol, start, end):
    df = df.set_index('timestamp')
    data = get_ohlc_data("1D", symbol, start, end)
    trade_df = pd.DataFrame(data, columns = ['timestamp', 'Open', 'Close', 'High', 'Low', 'Volume'])
    trade_df = trade_df.set_index('timestamp')
    df = pd.concat([df, trade_df])
    df = df[~df.index.duplicated()]
    return df
        

In [9]:
data = fetch_data_from_bitfinex("15m", "ETHUSD", 1605999600000, 1621720800000)